In [1]:
@file:Repository("https://repo.kotlin.link")
@file:DependsOn("space.kscience:kmath-tensors-jvm:0.3.0-dev-8")

In [2]:
import space.kscience.kmath.tensors.core.*
import space.kscience.kmath.operations.invoke

In [3]:
val randSeed = 100500L

# PCA

Take `x` values from range 0 to 9.

In [4]:
val x = DoubleTensorAlgebra {
    fromArray(
        intArrayOf(10),
        (0 until 10).toList().map { it.toDouble() }.toDoubleArray()
    )
} 

x

DoubleTensor(
   [ 0.0      ,  1.0      ,  2.0      ,  3.0      ,  4.0      ,  5.0      ,  6.0      ,  7.0      ,  8.0      ,  9.0      ]
)

Define `y` as dependent on `x` with noise.

In [5]:
val y = DoubleTensorAlgebra { 2.0 * x + (3.0 + x.randomNormalLike(randSeed) * 1.5) }

y

DoubleTensor(
   [ 2.4858   ,  6.0337   ,  7.7462   ,  9.3974   ,  9.9228   ,  1.1071e+1,  1.3791e+1,  1.5351e+1,  1.6051e+1,  2.0488e+1]
)

Stack them into single `dataset` tensor:

In [6]:
val dataset = DoubleTensorAlgebra { stack(listOf(x, y)).transpose() }
dataset

DoubleTensor(
   [[ 0.0      ,  2.4858   ],
    [ 1.0      ,  6.0337   ],
    [ 2.0      ,  7.7462   ],
    [ 3.0      ,  9.3974   ],
    [ 4.0      ,  9.9228   ],
    [ 5.0      ,  1.1071e+1],
    [ 6.0      ,  1.3791e+1],
    [ 7.0      ,  1.5351e+1],
    [ 8.0      ,  1.6051e+1],
    [ 9.0      ,  2.0488e+1]]
)

Normalize `x` and `y` and save means and stds for further recovery

In [7]:
val xMean = DoubleTensorAlgebra { x.mean() }
val yMean = DoubleTensorAlgebra { y.mean() }

val xStd = DoubleTensorAlgebra { x.std() }
val yStd = DoubleTensorAlgebra { y.std() }

val xScaled = DoubleTensorAlgebra { (x - xMean) / xStd }
val yScaled = DoubleTensorAlgebra { (y - yMean) / yStd }

In [8]:
val mean = DoubleTensorAlgebra { 
    fromArray(
        intArrayOf(2),
        doubleArrayOf(xMean, yMean)
    ) 
}

val std = DoubleTensorAlgebra {
    fromArray(
        intArrayOf(2),
        doubleArrayOf(xStd, yStd)
    ) 
}

In [9]:
mean

DoubleTensor(
   [ 4.5      ,  1.1234e+1]
)

In [10]:
std

DoubleTensor(
   [ 3.0277   ,  5.304    ]
)

Compute covaration matrix and its eigenvector.

In [11]:
val covMatrix = DoubleTensorAlgebra { cov(listOf(xScaled, yScaled)) }

covMatrix

DoubleTensor(
   [[ 1.0      ,  9.8423e-1],
    [ 9.8423e-1,  1.0      ]]
)

In [12]:
val (_, evecs) = DoubleTensorAlgebra { covMatrix.symEig() }
val eigVec = DoubleTensorAlgebra { evecs[0] }

eigVec

DoubleTensor(
   [-7.0711e-1, -7.0711e-1]
)

Reduce dimension of `dataset`.

In [13]:
val datasetReduced = DoubleTensorAlgebra { eigVec dot stack(listOf(xScaled, yScaled)) }

datasetReduced

DoubleTensor(
   [ 2.2172   ,  1.5107   ,  1.0488   ,  5.9515e-1,  2.9156e-1, -9.505e-2 , -6.912e-1 , -1.1327   , -1.4597   , -2.2848   ]
)

In [14]:
dataset.shape.joinToString(", ", "(", ")")

(10, 2)

In [15]:
datasetReduced.shape.joinToString(", ", "(", ")")

(10)

Restore the 7th row of `dataset`.

In [16]:
val n = 7
val restored = BroadcastDoubleTensorAlgebra { (datasetReduced[n] dot eigVec.view(intArrayOf(1, 2))) * std + mean }

In [17]:
restored

DoubleTensor(
   [ 6.9251   ,  1.5482e+1]
)

In [18]:
DoubleTensorAlgebra { dataset[n] }

DoubleTensor(
   [ 7.0      ,  1.5351e+1]
)

# OLS with SVD

Take `alpha` from normal distribution.

In [19]:
val alpha = DoubleTensorAlgebra { 
    randomNormal(
        intArrayOf(5),
        randSeed
    ) + fromArray(
        intArrayOf(5),
        doubleArrayOf(1.0, 2.5, 3.4, 5.0, 10.1)
    ) 
}

alpha

DoubleTensor(
   [ 3.8081e-1,  3.7449   ,  4.7119   ,  5.6986   ,  9.4929   ]
)

Take sample of size 20 from normal distribution for `x`.

In [20]:
val x = DoubleTensorAlgebra { 
    randomNormal(
        intArrayOf(20, 5),
        randSeed
    )
}

Calculate `y` and add gaussian noise $N(0, 0.05)$

In [21]:
val y = DoubleTensorAlgebra { x dot alpha }
DoubleTensorAlgebra { y += y.randomNormalLike(randSeed) * 0.05 }

Compute SVD of `x`.

In [22]:
val (u, singValues, v) = DoubleTensorAlgebra { x.svd() }

We have to make sure the singular values of the matrix are not close to zero.

In [23]:
singValues

DoubleTensor(
   [ 6.9973   ,  4.8911   ,  4.2973   ,  3.5892   ,  2.5359   ]
)

${Σ}^{-1}$ matrix can be restored from singular values with `diagonalEmbedding()` function

In [24]:
val sigmaInv = DoubleTensorAlgebra { diagonalEmbedding(singValues.map{ if (abs(it) < 1e-3) 0.0 else 1.0/it }) }

sigmaInv

DoubleTensor(
   [[ 1.4291e-1,  0.0      ,  0.0      ,  0.0      ,  0.0      ],
    [ 0.0      ,  2.0445e-1,  0.0      ,  0.0      ,  0.0      ],
    [ 0.0      ,  0.0      ,  2.327e-1 ,  0.0      ,  0.0      ],
    [ 0.0      ,  0.0      ,  0.0      ,  2.7861e-1,  0.0      ],
    [ 0.0      ,  0.0      ,  0.0      ,  0.0      ,  3.9433e-1]]
)

Estimate `alpha` with OLS.

In [25]:
val alphaOLS = DoubleTensorAlgebra { v dot sigmaInv dot u.transpose() dot y }

alphaOLS

DoubleTensor(
   [ 3.9396e-1,  3.7266   ,  4.6911   ,  5.6899   ,  9.4909   ]
)

In [26]:
alpha

DoubleTensor(
   [ 3.8081e-1,  3.7449   ,  4.7119   ,  5.6986   ,  9.4929   ]
)

Figure out MSE of approximation.

In [27]:
fun mse(yTrue: DoubleTensor, yPred: DoubleTensor): Double = DoubleTensorAlgebra {
    require(yTrue.shape.size == 1)
    require(yTrue.shape contentEquals yPred.shape)

    val diff = yTrue - yPred
    ((diff dot diff) / yTrue.shape[0].toDouble()).value()
}

mse(alpha, alphaOLS)

2.0310797370107518E-4

# Solving a linear system using the LU decomposition

In [28]:
val trueX = DoubleTensorAlgebra { 
    fromArray(
        intArrayOf(4),
        doubleArrayOf(-2.0, 1.5, 6.8, -2.4)
    ) 
}

trueX

DoubleTensor(
   [-2.0      ,  1.5      ,  6.8      , -2.4      ]
)

In [29]:
val a = DoubleTensorAlgebra { 
    fromArray(
        intArrayOf(4, 4),
        doubleArrayOf(
            0.5, 10.5, 4.5, 1.0,
            8.5, 0.9, 12.8, 0.1,
            5.56, 9.19, 7.62, 5.45,
            1.0, 2.0, -3.0, -2.5
        )
    )
}

a

DoubleTensor(
   [[ 5.0e-1   ,  1.05e+1  ,  4.5      ,  1.0      ],
    [ 8.5      ,  9.0e-1   ,  1.28e+1  ,  1.0e-1   ],
    [ 5.56     ,  9.19     ,  7.62     ,  5.45     ],
    [ 1.0      ,  2.0      , -3.0      , -2.5      ]]
)

In [30]:
val b = DoubleTensorAlgebra { a dot trueX }

b

DoubleTensor(
   [ 4.295e+1 ,  7.115e+1 ,  4.1401e+1, -1.34e+1  ]
)

Calculate $P$, $L$ and $U$, such that $PA = LU$.

In [31]:
val (p, l, u) = DoubleTensorAlgebra { a.lu() }

In [32]:
p

DoubleTensor(
   [[ 0.0      ,  1.0      ,  0.0      ,  0.0      ],
    [ 1.0      ,  0.0      ,  0.0      ,  0.0      ],
    [ 0.0      ,  0.0      ,  0.0      ,  1.0      ],
    [ 0.0      ,  0.0      ,  1.0      ,  0.0      ]]
)

In [33]:
l

DoubleTensor(
   [[ 1.0      ,  0.0      ,  0.0      ,  0.0      ],
    [ 5.8824e-2,  1.0      ,  0.0      ,  0.0      ],
    [ 1.1765e-1,  1.8131e-1,  1.0      ,  0.0      ],
    [ 6.5412e-1,  8.2332e-1,  7.4013e-1,  1.0      ]]
)

In [34]:
u

DoubleTensor(
   [[ 8.5      ,  9.0e-1   ,  1.28e+1  ,  1.0e-1   ],
    [ 0.0      ,  1.0447e+1,  3.7471   ,  9.9412e-1],
    [ 0.0      ,  0.0      , -5.1852   , -2.692    ],
    [ 0.0      ,  0.0      ,  0.0      ,  6.5585   ]]
)

In [35]:
DoubleTensorAlgebra { p dot a }

DoubleTensor(
   [[ 8.5      ,  9.0e-1   ,  1.28e+1  ,  1.0e-1   ],
    [ 5.0e-1   ,  1.05e+1  ,  4.5      ,  1.0      ],
    [ 1.0      ,  2.0      , -3.0      , -2.5      ],
    [ 5.56     ,  9.19     ,  7.62     ,  5.45     ]]
)

In [36]:
DoubleTensorAlgebra { l dot u }

DoubleTensor(
   [[ 8.5      ,  9.0e-1   ,  1.28e+1  ,  1.0e-1   ],
    [ 5.0e-1   ,  1.05e+1  ,  4.5      ,  1.0      ],
    [ 1.0      ,  2.0      , -3.0      , -2.5      ],
    [ 5.56     ,  9.19     ,  7.62     ,  5.45     ]]
)

In [37]:
DoubleTensorAlgebra { (p dot a) eq (l dot u) }

true

$Ax = b;$

$PAx = Pb;$

$LUx = Pb;$

let $y = Ux$, then

$Ly = Pb$ -- this system can be easily solved, since the matrix L is lower triangular;

$Ux = y$ can be solved the same way, since the matrix U is upper triangular

This function returns solution `x` of a system `lx = b`, `l` should be lower triangular.

In [38]:
fun solveLT(l: DoubleTensor, b: DoubleTensor): DoubleTensor = DoubleTensorAlgebra {
    val n = l.shape[0]
    val x = zeros(intArrayOf(n))
    for (i in 0 until n) {
        x[intArrayOf(i)] = (b[intArrayOf(i)] -  l[i].dot(x).value()) / l[intArrayOf(i, i)]
    }
    x
}

val y = DoubleTensorAlgebra { solveLT(l, p dot b) }

Create `solveUT` for solving system with upper triangular matrix with using permutation matrix.

In [39]:
val revMat = DoubleTensorAlgebra { u.zeroesLike() }
val n = revMat.shape[0]
for (i in 0 until n) {
    revMat[intArrayOf(i, n - 1 - i)] = 1.0
}

fun solveUT(u: DoubleTensor, b: DoubleTensor): DoubleTensor = DoubleTensorAlgebra {
    revMat dot solveLT(
        revMat dot u dot revMat, revMat dot b
    ) 
}

val x = solveUT(u, y)

In [40]:
x

DoubleTensor(
   [-2.0      ,  1.5      ,  6.8      , -2.4      ]
)

In [41]:
trueX

DoubleTensor(
   [-2.0      ,  1.5      ,  6.8      , -2.4      ]
)